In [2]:
import joblib

import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.tree import ExtraTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report, fbeta_score, make_scorer
import xgboost as xgb


## Evaluation Methods

In order to find the optimal model, F_beta score is applied in cross validation. beta equals to 2 indicates that we have a higher weight on the recall when selecting models . In other words, type II error will be weigh higher.

When evaluating the test data, the main measure for deciding the optimal model will be the challenge metric (Total cost = Cost1 · `#type I failures` + Cost2 · `#type II failures`). Besides, F_beta score and classification reports of precision/recall will be applied when running the script as a reference for the model performance. Together with challenge metrics, F_beta scores are recorded as F2 scores.

`#type I failures` is the count of type I failures (false positive), which indicates no faulty systems were reported positive falsely and leads to unnecessary mechanical check. While `#type II failures` is the count of type II failures (false negative), which indicates these problematic systems were reported no failure. The costs are 10 and 500 respectively.

## Load data


In [3]:
data = pd.read_csv('../data/ida_2016_training_set_update.csv', na_values=['na'])

In [4]:
# Split into X and y
X, y = data.drop(['class'], axis=1), data['class']

### Data imputation

In [5]:
# Import pickled imputer; SimpleImputer(missing_values=np.nan, strategy='median')
with open('../models/imputer.pkl','rb') as pickled_file:
    imputer = joblib.load(pickled_file)

In [6]:
# Imputate missing values by median
X_imputed = pd.DataFrame(imputer.transform(X), columns=X.columns)

In [7]:
X_imputed.head()

,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,ag_003,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,76698.0,0.0,2.130706e+09,280.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1240520.0,493384.0,721044.0,469792.0,339156.0,157956.0,73224.0,0.0,0.0,0.0
1,33058.0,0.0,0.000000e+00,126.0,0.0,0.0,0.0,0.0,0.0,0.0,...,421400.0,178064.0,293306.0,245416.0,133654.0,81140.0,97576.0,1500.0,0.0,0.0
2,41040.0,0.0,2.280000e+02,100.0,0.0,0.0,0.0,0.0,0.0,0.0,...,277378.0,159812.0,423992.0,409564.0,320746.0,158022.0,95128.0,514.0,0.0,0.0
3,12.0,0.0,7.000000e+01,66.0,0.0,10.0,0.0,0.0,0.0,318.0,...,240.0,46.0,58.0,44.0,10.0,0.0,0.0,0.0,4.0,32.0
4,60874.0,0.0,1.368000e+03,458.0,0.0,0.0,0.0,0.0,0.0,0.0,...,622012.0,229790.0,405298.0,347188.0,286954.0,311560.0,433954.0,1218.0,0.0,0.0


### Scaling data

In [8]:
X_imputed_scaled = X_imputed.copy()

In [9]:
# Load pickled scaler; preprocessing.MinMaxScaler(feature_range = (0, 1))
with open('../models/scaler.pkl','rb') as pickled_file: 
    scaler = joblib.load(pickled_file)

In [10]:
X_imputed_scaled = scaler.transform(X_imputed_scaled)

### Feature selection

Features can be irrelevant to the target variable and therefore create noise when fitting the algorithm. Due to highly correlated features in the data exploration step, it would be interesting to check feature importance and see if reduced features can improve model performances.

ExtraTrees will be applied here to select features based on their importance. ExtraTrees is one extension of Random forest, which is also a well-known method to measure the feature importance. For the ExtraTrees method, a random value based on the empirical training set would be selected for the split. The strength of the randomness is could choose an appropriate parameter based on the problem specifics. This algorithm would result in a vector of input feature importance, ranged from 0 to 1.

In [12]:
# Define the ExtraTreesClassifier model for feature selection
forest = ExtraTreeClassifier(random_state=3)

# Fit forest using all Xs and y's
forest.fit(X_imputed_scaled, y)

ExtraTreeClassifier(random_state=3)

In [13]:
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]

In [14]:
columns = data.columns

print("Feature ranking:")
for f in range(X.shape[1]):
    print("%d. feature %d - %s (%f)" % (f + 1, indices[f], columns[indices[f]], importances[indices[f]]))

Feature ranking:
1. feature 24 - ao_000 (0.164575)
2. feature 81 - bt_000 (0.079826)
3. feature 49 - az_006 (0.022841)
4. feature 164 - ee_005 (0.020417)
5. feature 55 - ba_002 (0.018982)
6. feature 25 - ap_000 (0.016529)
7. feature 40 - ay_007 (0.016008)
8. feature 52 - az_009 (0.015659)
9. feature 22 - am_0 (0.014874)
10. feature 98 - cl_000 (0.014864)
11. feature 142 - dm_000 (0.014718)
12. feature 20 - ak_000 (0.014637)
13. feature 165 - ee_006 (0.014325)
14. feature 39 - ay_006 (0.013545)
15. feature 9 - ag_002 (0.010987)
16. feature 82 - bu_000 (0.010905)
17. feature 158 - ed_000 (0.009921)
18. feature 12 - ag_005 (0.009184)
19. feature 96 - cj_000 (0.008785)
20. feature 134 - de_000 (0.008404)
21. feature 74 - bm_000 (0.008352)
22. feature 84 - bx_000 (0.008155)
23. feature 79 - br_000 (0.008126)
24. feature 160 - ee_001 (0.007637)
25. feature 21 - al_000 (0.007342)
26. feature 58 - ba_005 (0.007320)
27. feature 8 - ag_001 (0.007221)
28. feature 99 - cm_000 (0.007183)
29. featur

After applying ExtraTrees on the features, we got all 170 importance scores. The detailed scores and the feature rankings are shown as above. 

From previous sections of XGBoost and Logistic Regression models, we can select two optimal approaches - Logistic Regression with grid search CV and XGBoost with ‘scale_pos_weight’ equals to 59. In this experiment, we will use two sets of reduced features for training and evaluating these two optimal models - one set with features whose importance scores are larger than 0.005, the other set with importance scores that are larger than 0.01.

### Remove features that have importance scores less than 0.005

In [16]:
indices[53:] 

array([ 44, 135,  80,  23,  88, 159, 131,  97,  10, 152, 104,  90, 147,
       119,  87,  67, 163, 167, 128,  32, 149,  60,  34,  53,  83, 156,
       161,  13, 102,  64, 107,  57, 136,  70,  68, 125,  30, 143,  11,
        48,  46,  95,  56,  28, 162,   3,  61, 120,  15, 109,  94, 157,
       144,  16, 110,  19, 121,  54, 108,  65,  45,  92, 105, 114,  29,
        38,  14,  75, 106,  72, 103,  78, 146,  73,  51,  41, 123,  62,
        50,  26,  36, 137,   5,   1,  33,  35, 124, 133,  47,   2, 132,
       145, 127,   6, 112, 151,  31,  63,  91,  86, 168, 155, 126,   4,
       139, 130, 129,  27, 154, 153, 140, 122, 138,  89,  93, 141, 169])

In [17]:
importances[indices[53:]]

array([0.0048344 , 0.00475843, 0.00474338, 0.00461557, 0.00461312,
       0.00451327, 0.00443795, 0.00441762, 0.00437936, 0.0043751 ,
       0.00436766, 0.00432342, 0.00429241, 0.00428164, 0.00426892,
       0.00423991, 0.00418961, 0.00416185, 0.00414792, 0.00413232,
       0.00412034, 0.0040471 , 0.00400137, 0.00396809, 0.00392587,
       0.00387013, 0.0038654 , 0.00384064, 0.00382504, 0.00372782,
       0.00370615, 0.00368909, 0.00355614, 0.00354263, 0.00349789,
       0.00341123, 0.00341025, 0.00339257, 0.00335813, 0.00334912,
       0.00330861, 0.0032489 , 0.00320097, 0.00316659, 0.00312591,
       0.00311014, 0.003046  , 0.00298344, 0.00297381, 0.00270455,
       0.00263384, 0.00258637, 0.00256436, 0.00256414, 0.00255631,
       0.00253322, 0.00250537, 0.00250287, 0.00242797, 0.00242353,
       0.00239558, 0.00235547, 0.00232542, 0.0023002 , 0.00224042,
       0.00222781, 0.0022155 , 0.00216884, 0.00213165, 0.00211113,
       0.00203299, 0.00197213, 0.00192901, 0.00192761, 0.00191

In [18]:
removed_features = columns[indices[53:]]
removed_features

Index(['az_001', 'df_000', 'bs_000', 'an_000', 'cb_000', 'ee_000', 'db_000',
       'ck_000', 'ag_003', 'dx_000',
       ...
       'ar_000', 'dz_000', 'dy_000', 'dk_000', 'cs_008', 'di_000', 'cc_000',
       'cg_000', 'dl_000', 'ef_000'],
      dtype='object', length=117)

In [19]:
# Restructure the dataset by removing these less importatnt features
X_new = np.delete(X_imputed_scaled, indices[53:] , axis=1)

In [20]:
X_new.shape

(60000, 53)

In [21]:
# Split into training/testing dataset
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=3, stratify=y)

### Grid search to find best parameter C

In [29]:
# Function to get the total cost (competition metric)
def get_total_cost(type1_error, type2_error, cost1 = 10,cost2 = 500):
    # type II error: FN, cost=500
    # type I error: FP, cost=10
    return type1_error*cost1 + type2_error*cost2

# Function to get prettified confusion matrix
def get_confusion_matrix(y_pred, y_true=y_test):
    confusion_matrix_df = pd.DataFrame(
        confusion_matrix(y_true, y_pred, labels=['pos', 'neg']),
        index=['True:pos', 'True:neg'], 
        columns=['Pred:pos', 'Pred:neg']
    )
    
    return confusion_matrix_df

In [22]:
# Define a 5-fold splits in applying CV, random_state is fixed
fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=3)

In [23]:
# A grid of parameter 'C' in logistic regresssion
params_grid = {'C': np.power(10.0, np.arange(-5, 5))}

In [24]:
lr_model = LogisticRegression(random_state=3, class_weight='balanced', max_iter=3000, verbose=1)

ftwo_scorer = make_scorer(fbeta_score, beta=2, pos_label='pos')
grid_search_cv_models = GridSearchCV(lr_model, params_grid, cv=fold, scoring=ftwo_scorer)

In [25]:
# Fit the CV model for logistic regression
grid_search_cv_models.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=3, shuffle=True),
             estimator=LogisticRegression(class_weight='balanced',
                                          max_iter=3000, random_state=3,
                                          verbose=1),
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04])},
             scoring=make_scorer(fbeta_score, beta=2, pos_label=pos))

In [26]:
print(f'The best score during training: {grid_search_cv_models.best_score_}, with params: {grid_search_cv_models.best_params_}')

The best score during training: 0.6787490984756834, with params: {'C': 10.0}


In [27]:
y_pred = grid_search_cv_models.predict(X_test)

In [30]:
conf_mat = get_confusion_matrix(y_pred)
print(f'Confusion matrix:\n {conf_mat}')

Confusion matrix:
           Pred:pos  Pred:neg
True:pos       189        11
True:neg       317     11483


In [31]:
total_cost = get_total_cost(type1_error=317, type2_error=11)
print(f'Total cost: {total_cost}')

Total cost: 8670


In [32]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         neg       1.00      0.97      0.99     11800
         pos       0.37      0.94      0.54       200

    accuracy                           0.97     12000
   macro avg       0.69      0.96      0.76     12000
weighted avg       0.99      0.97      0.98     12000



In [33]:
ftwo_score = fbeta_score(y_test, y_pred, beta=2, pos_label='pos')

print(f'F2 score: {ftwo_score}')

F2 score: 0.7235834609494639


#### XGboost

'scale_pos_weight' value is used to scale the gradient for the positive class.

In [34]:
# estimate scale_pos_weight value
estimate = 59

In [37]:
# define model
xgb_model = xgb.XGBClassifier(objective="binary:logistic", eval_metric='aucpr', random_state=3, scale_pos_weight=estimate)

In [ ]:
def f_beta_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    beta = 2 # Less weight on precision, more weight on recall
    return 'ftwo_score', fbeta_score(y_pred, y_true, beta)

In [38]:
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='aucpr',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, random_state=3, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=59, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [39]:
y_pred2 = xgb_model.predict(X_test)

In [40]:
conf_mat2 = get_confusion_matrix(y_pred2)
print(f'Confusion matrix:\n {conf_mat2}')

Confusion matrix:
           Pred:pos  Pred:neg
True:pos       157        43
True:neg        35     11765


In [41]:
total_cost2 = get_total_cost(type1_error=35, type2_error=43)
print(f'Total cost: {total_cost2}')

Total cost: 21850


In [42]:
print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

         neg       1.00      1.00      1.00     11800
         pos       0.82      0.79      0.80       200

    accuracy                           0.99     12000
   macro avg       0.91      0.89      0.90     12000
weighted avg       0.99      0.99      0.99     12000



In [44]:
ftwo_score2 = fbeta_score(y_test, y_pred2, beta=2, pos_label='pos')

print(f'F2 score: {ftwo_score2}')

F2 score: 0.7913306451612905


### Remove features that have importance scores less than 0.01.

In [15]:
indices[16:] 

array([158,  12,  96, 134,  74,  84,  79, 160,  21,  58,   8,  99,  43,
       100,   7, 115, 118,   0,  66,  77,  85, 117, 113, 150,  42, 111,
        18,  69,  71, 166, 148,  76,  17,  59,  37, 101, 116,  44, 135,
        80,  23,  88, 159, 131,  97,  10, 152, 104,  90, 147, 119,  87,
        67, 163, 167, 128,  32, 149,  60,  34,  53,  83, 156, 161,  13,
       102,  64, 107,  57, 136,  70,  68, 125,  30, 143,  11,  48,  46,
        95,  56,  28, 162,   3,  61, 120,  15, 109,  94, 157, 144,  16,
       110,  19, 121,  54, 108,  65,  45,  92, 105, 114,  29,  38,  14,
        75, 106,  72, 103,  78, 146,  73,  51,  41, 123,  62,  50,  26,
        36, 137,   5,   1,  33,  35, 124, 133,  47,   2, 132, 145, 127,
         6, 112, 151,  31,  63,  91,  86, 168, 155, 126,   4, 139, 130,
       129,  27, 154, 153, 140, 122, 138,  89,  93, 141, 169])

In [46]:
importances[indices[16:]]

array([0.00992092, 0.00918449, 0.00878533, 0.00840436, 0.00835163,
       0.00815548, 0.00812579, 0.00763657, 0.00734242, 0.00732035,
       0.00722092, 0.00718279, 0.00693032, 0.00692643, 0.00688676,
       0.00684529, 0.00669871, 0.00662625, 0.00639502, 0.00636692,
       0.0062915 , 0.00623109, 0.00609775, 0.0059425 , 0.00583507,
       0.00571067, 0.00561239, 0.00559615, 0.00552748, 0.0054403 ,
       0.00533179, 0.00533009, 0.00527272, 0.00524911, 0.00520385,
       0.00507917, 0.00506447, 0.0048344 , 0.00475843, 0.00474338,
       0.00461557, 0.00461312, 0.00451327, 0.00443795, 0.00441762,
       0.00437936, 0.0043751 , 0.00436766, 0.00432342, 0.00429241,
       0.00428164, 0.00426892, 0.00423991, 0.00418961, 0.00416185,
       0.00414792, 0.00413232, 0.00412034, 0.0040471 , 0.00400137,
       0.00396809, 0.00392587, 0.00387013, 0.0038654 , 0.00384064,
       0.00382504, 0.00372782, 0.00370615, 0.00368909, 0.00355614,
       0.00354263, 0.00349789, 0.00341123, 0.00341025, 0.00339

In [47]:
removed_features2 = columns[indices[16:]]
removed_features2

Index(['ed_000', 'ag_005', 'cj_000', 'de_000', 'bm_000', 'bx_000', 'br_000',
       'ee_001', 'al_000', 'ba_005',
       ...
       'ar_000', 'dz_000', 'dy_000', 'dk_000', 'cs_008', 'di_000', 'cc_000',
       'cg_000', 'dl_000', 'ef_000'],
      dtype='object', length=154)

In [48]:
# Restructure the dataset by removing these less importatnt features
X_new2 = np.delete(X_imputed_scaled, indices[16:] , axis=1)

In [49]:
X_new2.shape

(60000, 16)

In [51]:
# Split into training/testing dataset
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_new2, y, test_size=0.2, random_state=3, stratify=y)

### Grid search to find best parameter C

In [52]:
lr_model2 = LogisticRegression(random_state=3, class_weight='balanced', max_iter=3000, verbose=1)

grid_search_cv_models2 = GridSearchCV(lr_model2, params_grid, cv=fold, scoring=ftwo_scorer)

In [53]:
# Fit the CV model for logistic regression
grid_search_cv_models2.fit(X_train2, y_train2)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=3, shuffle=True),
             estimator=LogisticRegression(class_weight='balanced',
                                          max_iter=3000, random_state=3,
                                          verbose=1),
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04])},
             scoring=make_scorer(fbeta_score, beta=2, pos_label=pos))

In [54]:
print(f'The best score during training: {grid_search_cv_models2.best_score_}, with params: {grid_search_cv_models2.best_params_}')

The best score during training: 0.6503491260753078, with params: {'C': 100.0}


In [56]:
y_pred3 = grid_search_cv_models2.predict(X_test2)

In [57]:
conf_mat3 = get_confusion_matrix(y_pred3)
print(f'Confusion matrix:\n {conf_mat3}')

Confusion matrix:
           Pred:pos  Pred:neg
True:pos       187        13
True:neg       347     11453


In [58]:
total_cost3 = get_total_cost(type1_error=347, type2_error=13)
print(f'Total cost: {total_cost3}')

Total cost: 9970


In [59]:
print(classification_report(y_test, y_pred3))

              precision    recall  f1-score   support

         neg       1.00      0.97      0.98     11800
         pos       0.35      0.94      0.51       200

    accuracy                           0.97     12000
   macro avg       0.67      0.95      0.75     12000
weighted avg       0.99      0.97      0.98     12000



In [60]:
ftwo_score3 = fbeta_score(y_test, y_pred3, beta=2, pos_label='pos')

print(f'F2 score: {ftwo_score3}')

F2 score: 0.7008995502248877


#### XGboost

In [61]:
# define model
xgb_model2 = xgb.XGBClassifier(objective="binary:logistic", eval_metric='aucpr', random_state=3, scale_pos_weight=estimate)

In [62]:
xgb_model2.fit(X_train2, y_train2)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='aucpr',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, random_state=3, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=59, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [63]:
y_pred4 = xgb_model2.predict(X_test2)

In [64]:
conf_mat4 = get_confusion_matrix(y_pred4)
print(f'Confusion matrix:\n {conf_mat4}')

Confusion matrix:
           Pred:pos  Pred:neg
True:pos       156        44
True:neg        62     11738


In [66]:
total_cost4 = get_total_cost(type1_error=62, type2_error=44)
print(f'Total cost: {total_cost4}')

Total cost: 22620


In [67]:
print(classification_report(y_test, y_pred4))

              precision    recall  f1-score   support

         neg       1.00      0.99      1.00     11800
         pos       0.72      0.78      0.75       200

    accuracy                           0.99     12000
   macro avg       0.86      0.89      0.87     12000
weighted avg       0.99      0.99      0.99     12000



In [68]:
ftwo_score4 = fbeta_score(y_test, y_pred4, beta=2, pos_label='pos')

print(f'F2 score: {ftwo_score4}')

F2 score: 0.7662082514734776
